# Preparing the Message Schedule

# We've gone as far as
1. converting the message
2. initializing the hash values
3. shifting and rotation
4. NOW Preparing the message

#### Full Algorithm

In [ ]:
import struct
import math

# Constants for SHA-256
K = [
    0x428a2f98, 0x71374491, 0xb5c0fbcf, 0xe9b5dba5, 0x3956c25b, 0x59f111f1, 0x923f82a4, 0xab1c5ed5,
    0xd807aa98, 0x12835b01, 0x243185be, 0x550c7dc3, 0x72be5d74, 0x80deb1fe, 0x9bdc06a7, 0xc19bf174,
    0xe49b69c1, 0xefbe4786, 0x0fc19dc6, 0x240ca1cc, 0x2de92c6f, 0x4a7484aa, 0x5cb0a9dc, 0x76f988da,
    0x983e5152, 0xa831c66d, 0xb00327c8, 0xbf597fc7, 0xc6e00bf3, 0xd5a79147, 0x06ca6351, 0x14292967,
    0x27b70a85, 0x2e1b2138, 0x4d2c6dfc, 0x53380d13, 0x650a7354, 0x766a0abb, 0x81c2c92e, 0x92722c85,
    0xa2bfe8a1, 0xa81a664b, 0xc24b8b70, 0xc76c51a3, 0xd192e819, 0xd6990624, 0xf40e3585, 0x106aa070,
    0x19a4c116, 0x1e376c08, 0x2748774c, 0x34b0bcb5, 0x391c0cb3, 0x4ed8aa4a, 0x5b9cca4f, 0x682e6ff3,
    0x748f82ee, 0x78a5636f, 0x84c87814, 0x8cc70208, 0x90befffa, 0xa4506ceb, 0xbef9a3f7, 0xc67178f2
]

# Initial hash values
H = [
    0x6a09e667, 0xbb67ae85, 0x3c6ef372, 0xa54ff53a,
    0x510e527f, 0x9b05688c, 0x1f83d9ab, 0x5be0cd19
]

def right_rotate(value, shift):
    return (value >> shift) | (value << (32 - shift)) & 0xFFFFFFFF

def pad_message(message):
    # Convert message to bytes using ASCII encoding
    message_bytes = message.encode('ascii')
    
    # Calculate the length of the original message in bits
    original_length_bits = len(message_bytes) * 8
    
    # Pad the message with a single '1' bit followed by '0' bits
    padded_message = message_bytes + b'\x80'
    while (len(padded_message) * 8) % 512 != 448:
        padded_message += b'\x00'
    
    # Append the length of the original message as a 64-bit big-endian integer
    padded_message += struct.pack('>Q', original_length_bits)
    
    return padded_message

def prepare_message_schedule(block):
    W = [0] * 64
    for t in range(16):
        W[t] = struct.unpack('>I', block[t * 4:t * 4 + 4])[0]
    for t in range(16, 64):
        s0 = (right_rotate(W[t-15], 7) ^ right_rotate(W[t-15], 18) ^ (W[t-15] >> 3))
        s1 = (right_rotate(W[t-2], 17) ^ right_rotate(W[t-2], 19) ^ (W[t-2] >> 10))
        W[t] = (W[t-16] + s0 + W[t-7] + s1) & 0xFFFFFFFF
    return W

def sha256_compress(block, H):
    W = prepare_message_schedule(block)
    a, b, c, d, e, f, g, h = H

    for t in range(64):
        S1 = right_rotate(e, 6) ^ right_rotate(e, 11) ^ right_rotate(e, 25)
        ch = (e & f) ^ (~e & g)
        temp1 = (h + S1 + ch + K[t] + W[t]) & 0xFFFFFFFF
        S0 = right_rotate(a, 2) ^ right_rotate(a, 13) ^ right_rotate(a, 22)
        maj = (a & b) ^ (a & c) ^ (b & c)
        temp2 = (S0 + maj) & 0xFFFFFFFF

        h = g
        g = f
        f = e
        e = (d + temp1) & 0xFFFFFFFF
        d = c
        c = b
        b = a
        a = (temp1 + temp2) & 0xFFFFFFFF

    H[0] = (H[0] + a) & 0xFFFFFFFF
    H[1] = (H[1] + b) & 0xFFFFFFFF
    H[2] = (H[2] + c) & 0xFFFFFFFF
    H[3] = (H[3] + d) & 0xFFFFFFFF
    H[4] = (H[4] + e) & 0xFFFFFFFF
    H[5] = (H[5] + f) & 0xFFFFFFFF
    H[6] = (H[6] + g) & 0xFFFFFFFF
    H[7] = (H[7] + h) & 0xFFFFFFFF

def sha256(message):
    # Pad the message
    padded_message = pad_message(message)
    
    # Process each 512-bit block
    for i in range(0, len(padded_message), 64):
        block = padded_message[i:i+64]
        sha256_compress(block, H)
    
    # Produce the final hash
    return ''.join(f'{value:08x}' for value in H)

# Example usage
message = "Hi!"
hash_value = sha256(message)
print(hash_value)

### Recap

Our original message.

In [25]:
sha_256_input = 'Hi!'
sha_256_input

'Hi!'

Message transformed to binary.

In [26]:
binary_message = text_to_binary(sha_256_input)
binary_message

'010010000110100100100001'

Then we pad the message with zeros, adding
- `1` to indicate the beginning of padding
- enough zeros `000000...` to make the padded message length a multiple of 512 *(512, 1024, 2056, etc.)*
- and a final 64-bit field indicating the length of the message in binary.

In [27]:
padded_message = pad_binary(binary_message)
padded_message

print(f"PADDED MESSAGE: {padded_message}")
print(f"LENGTH {len(padded_message)}")
print(f"MULTIPLE OF 512? {len(padded_message) % 512 == 0}")


PADDED MESSAGE: 01001000011010010010000110000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000011000
LENGTH 512
MULTIPLE OF 512? True


Visualized like this:

In [28]:
visualize_blocks(padded_message)


Block 1 (512 bits):
Bits   0-63 : 0100100001101001001000011000000000000000000000000000000000000000
Bits  64-127: 0000000000000000000000000000000000000000000000000000000000000000
Bits 128-191: 0000000000000000000000000000000000000000000000000000000000000000
Bits 192-255: 0000000000000000000000000000000000000000000000000000000000000000
Bits 256-319: 0000000000000000000000000000000000000000000000000000000000000000
Bits 320-383: 0000000000000000000000000000000000000000000000000000000000000000
Bits 384-447: 0000000000000000000000000000000000000000000000000000000000000000
Bits 448-511: 0000000000000000000000000000000000000000000000000000000000011000


### Prepare Message Schedule

In [103]:

def prepare_message_schedule(padded_message):
    # Split the padded message into 512-bit blocks
    blocks = [padded_message[i:i+512] for i in range(0, len(padded_message), 512)]
    
    # Initialize the message schedule array
    message_schedule = []

    for block in blocks:
        # Initialize the array W with the first 16 words
        W = [int(block[i:i+32], 2) for i in range(0, 512, 32)]
        
        # Extend the first 16 words into the remaining 48 words
        for t in range(16, 64):
            s0 = rightrotate(W[t-15], 7) ^ rightrotate(W[t-15], 18) ^ (W[t-15] >> 3)
            s1 = rightrotate(W[t-2], 17) ^ rightrotate(W[t-2], 19) ^ (W[t-2] >> 10)
            W.append((W[t-16] + s0 + W[t-7] + s1) & 0xFFFFFFFF)
        
        message_schedule.append(W)
    
    return message_schedule

# Prepare the message schedule for the padded message
message_schedule = prepare_message_schedule(padded_message)

# Print the first block's message schedule for verification
for i, word in enumerate(message_schedule[0]):
    print(f"W[{i:02}]: {word:032b}")

W[00]: 01001000011010010010000110000000
W[01]: 00000000000000000000000000000000
W[02]: 00000000000000000000000000000000
W[03]: 00000000000000000000000000000000
W[04]: 00000000000000000000000000000000
W[05]: 00000000000000000000000000000000
W[06]: 00000000000000000000000000000000
W[07]: 00000000000000000000000000000000
W[08]: 00000000000000000000000000000000
W[09]: 00000000000000000000000000000000
W[10]: 00000000000000000000000000000000
W[11]: 00000000000000000000000000000000
W[12]: 00000000000000000000000000000000
W[13]: 00000000000000000000000000000000
W[14]: 00000000000000000000000000000000
W[15]: 00000000000000000000000000011000
W[16]: 01001000011010010010000110000000
W[17]: 00000000000011110000000000000000
W[18]: 10110100111000100011011101110001
W[19]: 01100000000000000000001111000110
W[20]: 01011101011110111101010001100000
W[21]: 00000001100000111111110000000000
W[22]: 10010000101010110111101111111111
W[23]: 11001001111010011000000110001110
W[24]: 11010010101100110001000010011110
